In [6]:
import FluidSim
import numpy as np

import plotly.figure_factory as ff
import plotly.graph_objects as go



In [7]:
sim = FluidSim.FluidSim(41,41,2,2)
sim.timestep()
frames = []
#TODO https://plotly.com/python/carpet-contour/

ValueError: The first argument to the plotly.graph_objs.Frame 
constructor must be a dict or 
an instance of :class:`plotly.graph_objs.Frame`